In [1]:
import pandas as pd

# Utilizando multi indices
- Até o momento utilizamos dataframes com apenas um indice
- Mas é possível utilizar multiplos indices

In [2]:
bigmac = pd.read_csv("../data/bigmac.csv", parse_dates=["Date"])
bigmac.head()

/tmp/ipykernel_79186/3286211701.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  bigmac = pd.read_csv("../data/bigmac.csv", parse_dates=["Date"])


,Date,Country,Price in US Dollars
0,2016-01-01,Argentina,2.39
1,2016-01-01,Australia,3.74
2,2016-01-01,Brazil,3.35
3,2016-01-01,Britain,4.22
4,2016-01-01,Canada,4.14


- Para setarmos os indices, utilizamos o método `set_index()`
    - Uma outra maneira seria usando o parametro `index_col` quando usamos o método `read_csv`
- Esse método aceita uma ou várias colunas em formato de lista para se tornar o(s) indice(s)

In [3]:
bigmac.set_index(["Date", "Country"])

Price in US Dollars
Date       Country                           
2016-01-01 Argentina                     2.39
           Australia                     3.74
           Brazil                        3.35
           Britain                       4.22
           Canada                        4.14
...                                       ...
2010-01-01 Turkey                        3.83
           UAE                           2.99
           Ukraine                       1.83
           United States                 3.58
           Uruguay                       3.32

[652 rows x 1 columns]

- Perceba que ouve uma mesclagem entre `Date` e `Country`
- **Importante**: a ordem das colunas importam, pois afeta nessa agregação. Se trocarmos a ordem, perdemos esse comportamento

In [4]:
bigmac.set_index(["Country", "Date"])

,,Price in US Dollars
Country,Date,
Argentina,2016-01-01,2.39
Australia,2016-01-01,3.74
Brazil,2016-01-01,3.35
Britain,2016-01-01,4.22
Canada,2016-01-01,4.14
...,...,...
Turkey,2010-01-01,3.83
UAE,2010-01-01,2.99
Ukraine,2010-01-01,1.83


- Via de regra, utilizamos o label com menos ocorrência como o primeiro da lista para termos essa mesclagem de maneira a trazer mais informação

In [5]:
bigmac.set_index(["Date", "Country"], inplace=True)
bigmac.head()

Price in US Dollars
Date       Country                       
2016-01-01 Argentina                 2.39
           Australia                 3.74
           Brazil                    3.35
           Britain                   4.22
           Canada                    4.14

- Por padrão, se usarmos um método que use o indice, como o `sort_index()`, ele sempre começa a operação com o indece mais interno
    - Nesse caso, ele ordena por data e depois por país

In [7]:
bigmac.sort_index(inplace=True)
bigmac.head()

Price in US Dollars
Date       Country                       
2010-01-01 Argentina                 1.84
           Australia                 3.98
           Brazil                    4.76
           Britain                   3.67
           Canada                    3.97

- Outro ponto importante é que se tentarmos obter um indice, recebemos um objeto `MultiIndex`:
    - Que nada mais é do que um lista de tuplas de dois valores
    - Isso significa que para acessar um valor temos que combinar dois indices

In [8]:
bigmac.index

MultiIndex([('2010-01-01',      'Argentina'),
            ('2010-01-01',      'Australia'),
            ('2010-01-01',         'Brazil'),
            ('2010-01-01',        'Britain'),
            ('2010-01-01',         'Canada'),
            ('2010-01-01',          'Chile'),
            ('2010-01-01',          'China'),
            ('2010-01-01',       'Colombia'),
            ('2010-01-01',     'Costa Rica'),
            ('2010-01-01', 'Czech Republic'),
            ...
            ('2016-01-01',    'Switzerland'),
            ('2016-01-01',         'Taiwan'),
            ('2016-01-01',       'Thailand'),
            ('2016-01-01',         'Turkey'),
            ('2016-01-01',            'UAE'),
            ('2016-01-01',        'Ukraine'),
            ('2016-01-01',  'United States'),
            ('2016-01-01',        'Uruguay'),
            ('2016-01-01',      'Venezuela'),
            ('2016-01-01',        'Vietnam')],
           names=['Date', 'Country'], length=652)

- Podemos obter todos os indices de um multi indice usando o método `get_level_values()`:
    - Podemos usar o nome da coluna (`Date`) ou a posição (`0`)

In [9]:
bigmac.index.get_level_values("Date")

DatetimeIndex(['2010-01-01', '2010-01-01', '2010-01-01', '2010-01-01',
               '2010-01-01', '2010-01-01', '2010-01-01', '2010-01-01',
               '2010-01-01', '2010-01-01',
               ...
               '2016-01-01', '2016-01-01', '2016-01-01', '2016-01-01',
               '2016-01-01', '2016-01-01', '2016-01-01', '2016-01-01',
               '2016-01-01', '2016-01-01'],
              dtype='datetime64[ns]', name='Date', length=652, freq=None)

- Podemos trocar o nome de um multiindice usando o método `set_names()`

In [10]:
bigmac.index.set_names(names=["Day", "Location"], inplace=True)
bigmac.head()

Price in US Dollars
Day        Location                      
2010-01-01 Argentina                 1.84
           Australia                 3.98
           Brazil                    4.76
           Britain                   3.67
           Canada                    3.97

- É possível inverter o nível dos indices através do método `swaplevel()`

In [17]:
bigmac.swaplevel("Location", "Day")

,,Price in US Dollars
Location,Day,
Argentina,2010-01-01,1.84
Australia,2010-01-01,3.98
Brazil,2010-01-01,4.76
Britain,2010-01-01,3.67
Canada,2010-01-01,3.97
...,...,...
Ukraine,2016-01-01,1.54
United States,2016-01-01,4.93
Uruguay,2016-01-01,3.74


## Explorando melhor o método `sort_index()`
- Podemos controlar como é feita a ordenação considerando cada indice manipulando os parametros do metodo
- Por exemplo, podemos ordenar de maneira decrescente `Day` e crescente `Location`
    - O segredo é que agora podemos passar uma lista de parametros para isso

In [11]:
bigmac.sort_index(ascending=[False, True])

Price in US Dollars
Day        Location                          
2016-01-01 Argentina                     2.39
           Australia                     3.74
           Austria                       3.76
           Belgium                       4.25
           Brazil                        3.35
...                                       ...
2010-01-01 Turkey                        3.83
           UAE                           2.99
           Ukraine                       1.83
           United States                 3.58
           Uruguay                       3.32

[652 rows x 1 columns]

- Também podemos ordenar usando apenas um level, porém se eu escolhar o level mais interno, ele pode perder a mesclagem dos indices. Por exemplo:

In [11]:
bigmac.sort_index(level="Location").head()

,,Price in US Dollars
Day,Location,
2010-01-01,Argentina,1.84
2010-07-01,Argentina,3.56
2011-07-01,Argentina,4.84
2012-01-01,Argentina,4.64
2012-07-01,Argentina,4.16


## Extraindo linhas usando multi indices
- Para acessar os dados vamos usar o `.loc[]` mas com um tupla que representa os indices

In [12]:
bigmac.loc[("2010-01-01", "Uruguay")]

Price in US Dollars    3.32
Name: (2010-01-01 00:00:00, Uruguay), dtype: float64

- Obviamente pode especificar a coluna na sequencia:
    - Que também pode ser uma lista caso exista varias colunas

In [13]:
bigmac.loc[("2010-01-01", "Uruguay"), "Price in US Dollars"]

3.32

- Podemos acessar usando o método mais externo também:
    - nesse caso, retorna todos os indices que estao mesclados nele

In [14]:
bigmac.loc[("2010-01-01",)]

,Price in US Dollars
Location,
Argentina,1.84
Australia,3.98
Brazil,4.76
Britain,3.67
Canada,3.97
Chile,3.18
China,1.83
Colombia,3.91
Costa Rica,3.52


- Também podemos usar o `iloc[]`, mas neste caso, a linha é contada a partir do zero normalmente:

In [15]:
bigmac.iloc[0]

Price in US Dollars    1.84
Name: (2010-01-01 00:00:00, Argentina), dtype: float64

In [16]:
bigmac.iloc[[5, 10, 100, 120]]

Price in US Dollars
Day        Location                     
2010-01-01 Chile                    3.18
           Denmark                  5.99
2011-07-01 Estonia                  3.15
           Norway                   8.31